In [1]:
import os

SUBSCRIPTION_ID = os.environ.get("AZURE_SUBSCRIPTION_ID")
RESOURCE_GROUP = os.environ.get("AZURE_RESOURCE_GROUP")
AML_WORKSPACE_NAME = os.environ.get("AZUREAI_PROJECT_NAME")
ONLINE_ENDPOINT_NAME = os.environ.get("AZUREAI_ENDPOINT_NAME")

In [2]:
from azure.identity import (
    DefaultAzureCredential,
    InteractiveBrowserCredential,
    AzureCliCredential,
)
from azure.ai.ml import (
    MLClient,
)

try:
    credential = DefaultAzureCredential(additionally_allowed_tenants=["*"])
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    credential = InteractiveBrowserCredential(additionally_allowed_tenants=["*"])

ml_client = MLClient(credential, SUBSCRIPTION_ID, RESOURCE_GROUP, AML_WORKSPACE_NAME)

In [3]:
!az account set --subscription {SUBSCRIPTION_ID}
!az configure --defaults workspace={AML_WORKSPACE_NAME} group={RESOURCE_GROUP}

In [4]:
from azure.ai.ml.entities import (
    ManagedOnlineEndpoint,
)

# get the online endpoint
endpoint = ManagedOnlineEndpoint(
    name=ONLINE_ENDPOINT_NAME
)

endpoint = ml_client.begin_create_or_update(endpoint).result()

In [5]:
# you can customize your inference experience following Langserve instruction. The below code is just a simple example.
from langserve import RemoteRunnable
token = ml_client.online_endpoints.get_keys(name=endpoint.name).primary_key
runnable_az = RemoteRunnable(f"{endpoint.scoring_uri}openai-functions-agent", headers={"Authorization": "Bearer " + token})
async for msg in runnable_az.astream({
    "chat_history": [],  
    "input": "what is the work from home policy from langchain-test-index?"
}):
    print(msg, end="", flush=True)

ModuleNotFoundError: No module named 'langserve'